In [ ]:
import pandas as pd 
import numpy as np
import os
import shutil
from sklearn.model_selection import train_test_split

selected_frames_path = '../../videos/results/selected_frames_b.txt'
dataset_path = '../../videos/results/dataset_ball_b/'
output_path = '../../videos/results/dataset_ball_b_filtered/'
df_selected = pd.read_csv(selected_frames_path, header=None, sep=' ')
selected = np.array(df_selected[1])

j = 0
folders = ['images/', 'labels/', 'masks/']
extensions = ['jpg', 'txt', 'png']

for folder in folders:
    os.makedirs(os.path.join(output_path, folder), exist_ok=True)

for i in range(len(selected)):
    if selected[i]:
        for folder, extension in zip(folders, extensions):
            namein = os.path.join(dataset_path, folder, f'{i:05d}.{extension}')
            nameout = os.path.join(output_path, folder, f'{j:05d}.{extension}')
            if os.path.exists(namein):
                shutil.copy(namein, nameout)
        j += 1

datasize = j

shutil.copy(os.path.join(dataset_path, 'data.yaml'), os.path.join(output_path, 'data.yaml'))


train_split = 0.8
val_split = 0.1
test_split = 0.1

random_state = 42
class_id = 32

# Divisiones
train_idx, temp_idx = train_test_split(range(datasize), test_size=1-train_split, random_state=random_state)
if val_split>0:
    val_idx, test_idx = train_test_split(temp_idx, test_size=(1-train_split-val_split)/(1-train_split), random_state=random_state+1)
else:
    val_idx = []
    test_idx = temp_idx

subsets = {'train': train_idx, 'val': val_idx, 'test': test_idx}

# data.yaml
with open(os.path.join(output_path, 'data.yaml'), "w") as f:
    f.write(f"""path: .\ntrain: images/train\nval: images/val\ntest: images/test\nnames:\n {class_id}: ball""")

for subset, indices in subsets.items():
    for folder, extension in zip(folders, extensions):
        os.makedirs(os.path.join(output_path, folder, subset), exist_ok=True)
    for idx in indices:
        for folder, extension in zip(folders, extensions):
            namein = os.path.join(output_path, folder, f'{idx:05d}.{extension}')
            nameout = os.path.join(output_path, folder, subset, f'{idx:05d}.{extension}')
            if os.path.exists(namein):
                shutil.move(namein, nameout)